<a href="https://colab.research.google.com/github/TravisJRCain/DS-Unit-4-Sprint-2-Neural-Networks/blob/master/module4-Hyperparameter-Tuning/Travis_Cain_LS_DS_434_Hyperparameter_Tuning_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [0]:
##### Your Code Here #####

# Import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam

In [4]:
# Import data

df = pd.read_csv('/content/WA_Fn-UseC_-Telco-Customer-Churn+(1) (2).csv')
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [5]:
df.shape

(7043, 21)

In [6]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [7]:
df.describe(exclude='number')

,customerID,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,TotalCharges,Churn
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4,6531,2
top,6366-ZGQGL,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,,No
freq,1,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365,11,5174


In [0]:
# Clean the data

encoder = OneHotEncoder()
label_encoder = LabelEncoder()

In [0]:
binary = ['gender', 'Partner', 'Dependents', 'PhoneService',
          'PaperlessBilling', 'Churn']

words = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
         'Contract', 'PaymentMethod', 'TotalCharges']

In [0]:
# run transformation on each column, make sure they are integers

for col in binary:
  df[col] = encoder.fit_transform(df[[col]]).todense()
  df[col] = df[col].astype(int)

In [0]:
for col in words:
  df[col] = label_encoder.fit_transform(df[col])
  df[col] = df[col].astype(int)

In [12]:
# Check data

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,1,0,0,1,1,1,1,0,0,2,0,0,0,0,0,0,2,29.85,2505,1
1,5575-GNVDE,0,0,1,1,34,0,0,0,2,0,2,0,0,0,1,1,3,56.95,1466,1
2,3668-QPYBK,0,0,1,1,2,0,0,0,2,2,0,0,0,0,0,0,3,53.85,157,0
3,7795-CFOCW,0,0,1,1,45,1,1,0,2,0,2,2,0,0,1,1,0,42.30,1400,1
4,9237-HQITU,1,0,1,1,2,0,0,1,0,0,0,0,0,0,0,0,2,70.70,925,0


In [13]:
df.shape

(7043, 21)

In [14]:
# Remove useless data, use churn for testing

X = df.drop(columns=['Churn', 'customerID', 'TotalCharges'])
y = df['Churn']

X.shape, y.shape

((7043, 18), (7043,))

In [0]:
# Scaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [16]:
# Fit a model

inputs = X.shape[1]
epochs = 30
batch_size = 42

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=inputs))
model.add(Dense(16, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                608       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 1,153
Trainable params: 1,153
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(X,y, validation_split=0.25,
          epochs=epochs,
          batch_size=batch_size)

Epoch 1/30
126/126 [==============================] - 0s 4ms/step - loss: 2.2658 - accuracy: 0.5895 - val_loss: 1.3697 - val_accuracy: 0.6899
Epoch 2/30
126/126 [==============================] - 0s 2ms/step - loss: 1.0708 - accuracy: 0.7117 - val_loss: 1.0966 - val_accuracy: 0.7195
Epoch 3/30
126/126 [==============================] - 0s 2ms/step - loss: 0.8324 - accuracy: 0.7285 - val_loss: 0.9354 - val_accuracy: 0.7308
Epoch 4/30
126/126 [==============================] - 0s 2ms/step - loss: 0.7268 - accuracy: 0.7363 - val_loss: 0.8035 - val_accuracy: 0.6979
Epoch 5/30
126/126 [==============================] - 0s 2ms/step - loss: 0.6416 - accuracy: 0.7083 - val_loss: 0.7477 - val_accuracy: 0.7342
Epoch 6/30
126/126 [==============================] - 0s 2ms/step - loss: 0.6067 - accuracy: 0.7378 - val_loss: 0.7162 - val_accuracy: 0.7405
Epoch 7/30
126/126 [==============================] - 0s 2ms/step - loss: 0.5883 - accuracy: 0.7425 - val_loss: 0.7115 - val_accuracy: 0.7479
Epoch 

In [18]:
model.evaluate(X, y)

221/221 [==============================] - 0s 1ms/step - loss: 0.5419 - accuracy: 0.7948


[0.5419068932533264, 0.7948317527770996]

In [19]:
### New Model ###

def create_model():
  model = Sequential()
  model.add(Dense(1, activation='selu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = KerasClassifier(build_fn=create_model, verbose=0)

param_grid = {'batch_size': [8, 16, 32, 64, 128],
              'epochs': [30]}

# Grid Search

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = grid.fit(X, y)

# Reveal obtained results

print(f'Best: {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.8051959991455078 using {'batch_size': 8, 'epochs': 30}
Means: 0.8051959991455078, Stdev: 0.007562775200077747 with: {'batch_size': 8, 'epochs': 30}
Means: 0.8036350011825562, Stdev: 0.008400089309438367 with: {'batch_size': 16, 'epochs': 30}
Means: 0.8033506274223328, Stdev: 0.009631775750906001 with: {'batch_size': 32, 'epochs': 30}
Means: 0.8020727276802063, Stdev: 0.006519209708236414 with: {'batch_size': 64, 'epochs': 30}
Means: 0.7868807077407837, Stdev: 0.005804288824143118 with: {'batch_size': 128, 'epochs': 30}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?